# Gerekli Kütüphaneler

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor as SKAdaBoostRegressor
from sklearn import datasets

seed = 42
np.random.seed(seed)

# Veri Seti

In [2]:
X, y = datasets.make_regression(
    n_samples=200,
    n_features=1,
    noise=15,
    random_state=seed
)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)

# Adaptive Boosting

In [11]:
class AdaBoostRegressor:
    def __init__(self, n_estimators=50):
        # Zayif ogrenici sayisi
        self.n_estimators = n_estimators
        # Zayif modeller
        self.models = []
        # Zayif modellerin katki orani
        self.alphas = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples

        for _ in range(self.n_estimators):
            stump = DecisionTreeRegressor(max_depth=1)
            stump.fit(X, y, sample_weight=w)

            y_pred = stump.predict(X)
            error = np.sum(w * np.abs(y - y_pred)) / np.sum(w)

            alpha = 0.5 * np.log((1 - error) / (error + 1e-10))

            self.alphas.append(alpha)
            self.models.append(stump)

            w *= np.exp(-alpha * (y - y_pred) ** 2)
            w /= np.sum(w)

    def predict(self, X):
        pred = np.sum([alpha * model.predict(X) for alpha, model in zip(self.alphas, self.models)], axis=0)
        return pred

# Eğitim

In [ ]:
ada = AdaBoostRegressor()
ada.fit(X_train, y_train)

# Tahmin

In [ ]:
y_pred = ada.predict(X_test)

In [ ]:
print("R^2 Score:", r2_score(y_test, y_pred))

In [ ]:
print("MSE Score:", mean_squared_error(y_test, y_pred))

# Sonuçlar

In [ ]:
plt.figure(figsize=(3, 4))
plt.plot(X_test, y_test, "rx")
plt.plot(X_test, y_pred, "b")
plt.show()